<a href="https://colab.research.google.com/github/iAmEthanMai/chess-engine-model/blob/main/train_chess_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Python Chess Engine Train

**libraries**

---

In [ ]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

**connect to google drive**

---

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**data**

---

import the chess games dataset from github

In [ ]:
!git clone https://github.com/iAmEthanMai/chess-games-dataset.git

Cloning into 'chess-games-dataset'...
remote: Enumerating objects: 1375, done.
remote: Counting objects: 100% (1375/1375), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1375 (delta 1351), reused 1349 (delta 1335), pack-reused 0
Receiving objects: 100% (1375/1375), 10.33 MiB | 9.63 MiB/s, done.
Resolving deltas: 100% (1351/1351), done.
Checking out files: 100% (1346/1346), done.


In [ ]:


path_fischer = '/content/chess-games-dataset/Data/CSV_FISCHER'
path_morphy = '/content/chess-games-dataset/Data/CSV_MORPHY'
path_capablanca = '/content/chess-games-dataset/Data/CSV_CAPABLANCA'

files_fischer = glob.glob(path_fischer + "/*.csv")
#files_morphy = glob.glob(path_morphy + "/*.csv")
#files_capablanca = glob.glob(path_capablanca + "/*.csv")

            
li = []

for filename in files_fischer:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

train = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
train = shuffle(train)

In [ ]:
train.shape

(830376, 193)

In [ ]:
train.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,c2,d2,e2,f2,g2,h2,a3,b3,c3,d3,e3,f3,g3,h3,a4,b4,c4,d4,e4,f4,g4,h4,a5,b5,c5,d5,e5,f5,g5,h5,...,to_b4,to_c4,to_d4,to_e4,to_f4,to_g4,to_h4,to_a5,to_b5,to_c5,to_d5,to_e5,to_f5,to_g5,to_h5,to_a6,to_b6,to_c6,to_d6,to_e6,to_f6,to_g6,to_h6,to_a7,to_b7,to_c7,to_d7,to_e7,to_f7,to_g7,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
544480,None,None,None,None,None,R,K,None,P,P,P,None,R,None,P,P,None,None,P,None,None,N,None,None,None,None,None,None,None,None,None,None,None,None,p,None,None,None,None,None,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
258976,R,None,None,None,R,None,K,None,P,P,None,None,B,None,P,P,None,None,None,None,P,None,None,None,None,None,None,None,q,None,None,None,None,p,Q,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
224381,R,None,B,Q,K,None,None,R,P,P,P,None,None,P,P,P,None,None,N,None,None,None,None,None,None,None,B,N,P,None,None,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
50461,R,None,None,K,None,None,None,None,P,P,None,None,B,r,None,None,None,N,P,None,None,None,P,None,None,None,None,None,None,None,None,None,None,None,None,None,p,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,False
106124,None,None,None,None,None,R,None,None,P,None,None,None,None,R,P,None,None,P,None,None,K,None,None,P,None,None,B,None,p,None,None,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


**features**

---

In [ ]:
features = list(train.iloc[:, 0:192].columns)

In [ ]:
X = train[features]
y = train['good_move']

In [ ]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

**input function**

---

In [ ]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

**split data into batches**

---

In [ ]:
def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []
  
  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)
  
  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)
  
  batches_X = []
  batches_y = []
  
  for i in range(0, len(intervals) - 1):
    batches_X.append(train.iloc[intervals[i]:intervals[i + 1], :][features])
    batches_y.append(train.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return batches_X, batches_y

In [ ]:
batches_X, batches_y = split_into_batches(train)

**model**

---

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='/content/gdrive/MyDrive/chess-engine/estimator')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/MyDrive/chess-engine/estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


**train model**

---

In [ ]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
  input_functions.append(make_input_fn(df_X, df_y))

In [ ]:
len(input_functions)

9

In [ ]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)
  

# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = '/content/gdrive/MyDrive/chess-engine/estimator'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)